# ID2222 Data Mining, Homework 3
# **Mining Data Streams**

Brando Chiminelli, Tommaso Praturlon

November 28th, 2022

## Goal
The goal of this notebook is to 

The improved version is based on the base version of TRIEST. The changes are:

1. UpdateCounters is called unconditionally for each element on the stream, before the algorithm decides whether or not to insert the edge into S.
2. TRIEST-impr never decrements the counters when an edge is removed from S.
3. UpdateCounters performs a weighted increase of the counters using η(t) = max{1,(t − 1)(t − 2)/(M(M − 1))} as weight.

## Import libraries and read the dataset
In the following we import the few libraries needed for the project and we read the dataset.

We decided to read the first 2000 baskets from the dataset in order to reduce weight on memory. Our assumption is that items are uniformly distributed across the dataset, thus allowing us to have a good insight only from the given sample.

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations
import statistics
import math

PATH_TO_DATA = "../data/web-Stanford.txt"
df_graphs = pd.read_csv(PATH_TO_DATA, header=None)
print("Data read successfully!")

# Reduce dataset size for computation overload (temporary)
df_graphs = df_graphs.iloc[0:50]
print(df_graphs.head())
print("Number of rows: ", len(df_graphs))

Data read successfully!
                                                   0
0  # Directed graph (each unordered pair of nodes...
1                     # Stanford web graph from 2002
2                     # Nodes: 281903 Edges: 2312497
3                             # FromNodeId\tToNodeId
4                                            1\t6548
Number of rows:  50


In [19]:
# DATA WRANGLING
# 1. remove text rows from dataset
# 2. create a dataset of integers
df_g = df_graphs.drop([0, 1, 2, 3, 4])
data = []
for i in range(len(df_g)):
    s = [int(x) for x in str(df_g.iloc[i][0]).split('\t')]
    #s = str(df_g.iloc[i][0]).split('\t')
    data.append(s)
print(data)

[[1, 15409], [6548, 57031], [15409, 13102], [2, 17794], [2, 25202], [2, 53625], [2, 54582], [2, 64930], [2, 73764], [2, 84477], [2, 98628], [2, 100193], [2, 102355], [2, 105318], [2, 105730], [2, 115926], [2, 140864], [2, 163550], [2, 164599], [2, 175799], [2, 178642], [2, 181714], [2, 190453], [2, 204189], [2, 204604], [2, 210870], [2, 213966], [2, 225119], [2, 241596], [2, 243294], [2, 246897], [2, 251658], [2, 252915], [2, 280935], [252915, 2], [246897, 2], [246897, 78056], [251658, 2], [280935, 2], [213966, 2], [213966, 47149], [243294, 2], [225119, 2], [241596, 2], [178642, 2]]


In [15]:
####################################
# TRIEST-BASE CLASS IMPLEMENTATION #
####################################

class TriestBase:
    '''
    Implementation of the Trièst-base algorithm
    - function SampleEdge
    - function UpdateCounter
    - function FlipBiasedCoin
    '''
        
    def flipBiasedCoin(self, M, t):
        '''
        Flip a biased coin with probability M/t of falling head.
        '''
        import numpy as np
        # 1: head, 0: tail
        result = np.random.choice([1, 0], p=[M/t, (1-M/t)])
        if result:
            return True
        else:
            return False
    
    def sampleEdge(self, edge, t, S, M, tau, tau_loc):
        '''
        Receives the edge (u,v) as [u, v] and time t at which
        the stream element is received. t is a count integer.
        Returns a boolean
        '''
        import random
        
        if (t <= M):
            return True
        elif self.flipBiasedCoin(M, t):
            # select random edge from S
            random_edge = random.choice(S)
            # Delete random_edge from S
            S.remove(random_edge)
            # IMPR: remove the call to function update_counters
            #self.updateCounters('delete', random_edge, S, tau, tau_loc)
            return True
        else:
            return False
            
    def updateCounters(self, operation, t, edge, S, M, tau, tau_loc):
        '''
        Receives the operation insertion or deletion
        and the edge.
        tau is the global counter
        e.g S = [[5,10], [10, 3], [12, 5], [12, 3]]
        edge = (5, 12)
        N_5 = (10, 12)
        N_12 = (5, 3)
        N_5_12 = (5, 12)
        '''

        # trièst-impr never decrements the counters when an edge is removed from S
        # operation only 'insert'
        # UpdateCounters performs a weighted increase of the counters

        # Define shared-neighborhood
        shared_neigh = set()
        neigh_u = set() # all neighbors of edge[0]
        neigh_v = set() # all nneighbors of edge[1]
        #print("S from updateCounters():\n", S)
        for elem in S:
            # check that for the v in V_t (u,v) belongs to S
            # create neigh_u
            if edge[0] == elem[0]: # found u in position 0
                neigh_u.add(edge[1]) # add the other element
            if edge[0] == elem[1]: # found u in position 1
                neigh_u.add(edge[0]) # add the other element
            # create neigh_v
            if edge[1] == elem[0]: # found v in position 0
                neigh_v.add(edge[1]) 
            if edge[1] == elem[1]: # found v in position 1
                neigh_v.add(edge[0])
        # shared neighbourhood is the intersection between the sets
        #shared_neigh = neigh_u.intersection(neigh_v)
        shared_neigh = set.intersection(neigh_u, neigh_v)

        weight = max(1, (((t-1)*(t-2))/(M*(M-1))))

        for c in shared_neigh:
                tau += weight
                tau_loc[c] = tau_loc.get(c, 0) + weight
                tau_loc[edge[0]] = tau_loc.get(edge[0], 0) + weight
                tau_loc[edge[1]] = tau_loc.get(edge[1], 0) + weight

        
        return tau, tau_loc

In [16]:
# EXPLOITING THE CLASS TRIEST-BASE

'''
At time _t
Graph G_t = (set of verteces V_t, set of edges E_t) 
'''

S = []
M = 3000
t = 0
tau = 0
tau_loc = {} # dictionary of local counts (u: counts) {5: 99, 12: 3, 2:1}

print("Number of edges: ", len(data))
# simulate the stream of data
for i in range(len(data)):
    t += 1
    edge = data[i]
    #print(edge)
    if TriestBase().sampleEdge(edge, t, S, M, tau, tau_loc):
        S.append(edge)

    tau, tau_loc = TriestBase().updateCounters('insert', t, edge, S, M, tau, tau_loc)
    
        
# Epsilon
print(t)
eps = max(1, ((t*(t-1)*(t-2))/(M*(M-1)*(M-2))))
est_triangles = eps*tau
print("Tau: ", tau)
#print("Tau_loc: ", tau_loc)
print("Eps: ", eps)
print("Estimated triangles: ", est_triangles)

Number of edges:  2312496


KeyboardInterrupt: 

In [ ]:
# Graphs and statistics 
import matplotlib.pyplot as plt
from collections import Counter
'''
cnt = Counter(C_1.values())
plt.bar(cnt.keys(), cnt.values())

plt.xlabel('Frequency')
plt.ylabel('Number of documents with that frequency')
plt.show()
'''

"\ncnt = Counter(C_1.values())\nplt.bar(cnt.keys(), cnt.values())\n\nplt.xlabel('Frequency')\nplt.ylabel('Number of documents with that frequency')\nplt.show()\n"